In [120]:
from datetime import timedelta
import numpy as np
import pandas as pd

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from tqdm.notebook import tqdm

In [121]:
df = pd.read_csv("dataset.txt")

In [122]:
df.head(10)

,date,hour est time,timestamps utc,id,duration in one minute(totalsecs),line number (160 or170),timestamps est,minute est,timezone est
0,2022-11-16,6,2022-11-16 11:44:00,325,60,160,2022-11-16 06:44:00,44,America/New_York
1,2022-11-16,10,2022-11-16 15:32:00,5734,14,160,2022-11-16 10:32:00,32,America/New_York
2,2022-11-16,7,2022-11-16 12:29:00,1437,60,160,2022-11-16 07:29:00,29,America/New_York
3,2022-11-16,12,2022-11-16 17:02:00,6676,16,160,2022-11-16 12:02:00,2,America/New_York
4,2022-11-16,8,2022-11-16 13:48:00,2525,60,160,2022-11-16 08:48:00,48,America/New_York
5,2022-11-16,13,2022-11-16 18:18:00,8003,15,160,2022-11-16 13:18:00,18,America/New_York
6,2022-11-16,9,2022-11-16 14:42:00,3866,49,160,2022-11-16 09:42:00,42,America/New_York
7,2022-11-16,14,2022-11-16 19:16:00,9267,55,160,2022-11-16 14:16:00,16,America/New_York
8,2022-11-16,10,2022-11-16 15:22:00,5112,21,160,2022-11-16 10:22:00,22,America/New_York
9,2022-11-16,7,2022-11-16 12:05:00,908,41,160,2022-11-16 07:05:00,5,America/New_York


# Data Exploratory Analysis

In [123]:
df.shape

(361942, 9)

In [124]:
df.columns

Index(['date', 'hour est time', 'timestamps utc', 'id',
       'duration in one minute(totalsecs)', 'line number (160 or170)',
       'timestamps est', 'minute est', 'timezone est'],
      dtype='object')

In [125]:
df.dtypes

date                                 object
hour est time                         int64
timestamps utc                       object
id                                    int64
duration in one minute(totalsecs)     int64
line number (160 or170)               int64
timestamps est                       object
minute est                            int64
timezone est                         object
dtype: object

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361942 entries, 0 to 361941
Data columns (total 9 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   date                               361942 non-null  object
 1   hour est time                      361942 non-null  int64 
 2   timestamps utc                     361942 non-null  object
 3   id                                 361942 non-null  int64 
 4   duration in one minute(totalsecs)  361942 non-null  int64 
 5   line number (160 or170)            361942 non-null  int64 
 6   timestamps est                     361942 non-null  object
 7   minute est                         361942 non-null  int64 
 8   timezone est                       361942 non-null  object
dtypes: int64(5), object(4)
memory usage: 24.9+ MB


In [127]:
df.describe()

,hour est time,id,duration in one minute(totalsecs),line number (160 or170),minute est
count,361942.000000,361942.000000,361942.000000,361942.000000,361942.000000
mean,9.929773,2293.245763,30.062485,164.533599,29.669265
std,2.359683,2936.969236,22.263440,4.978206,17.127742
min,6.000000,1.000000,0.000000,160.000000,0.000000
25%,8.000000,455.000000,8.000000,160.000000,15.000000
50%,10.000000,1239.000000,27.000000,160.000000,30.000000
75%,12.000000,3015.000000,53.000000,170.000000,44.000000
max,14.000000,32775.000000,60.000000,170.000000,59.000000


# Data Preprocessing

In [128]:
df.isna().sum()

date                                 0
hour est time                        0
timestamps utc                       0
id                                   0
duration in one minute(totalsecs)    0
line number (160 or170)              0
timestamps est                       0
minute est                           0
timezone est                         0
dtype: int64

Null values are not present in the given dataset.

In [129]:
df.duplicated().sum()

311

In [130]:
df.drop_duplicates(inplace=True)

Removing the duplicate observations as there are 311 observations which are duplicated. We droped the duplicates since our dataset is huge and 311 is very small observations compared to total observations.

In [131]:
df['date'] = pd.to_datetime(df['date'])
df['timestamps utc'] = pd.to_datetime(df['timestamps utc'])
df['timestamps est'] = pd.to_datetime(df['timestamps est'])

Converting the string datetime columns into pandas datetime columns.

In [132]:
selected_columns = ['timestamps utc', 'id', 'duration in one minute(totalsecs)', 'line number (160 or170)']
df_pro = df[selected_columns].sort_values(by=['line number (160 or170)', 'timestamps utc']).reset_index(drop=True)
df_pro

,timestamps utc,id,duration in one minute(totalsecs),line number (160 or170)
0,2022-01-04 12:17:00,14,8,160
1,2022-01-04 12:46:00,103,7,160
2,2022-01-04 12:47:00,112,8,160
3,2022-01-04 12:48:00,112,1,160
4,2022-01-04 12:49:00,112,8,160
...,...,...,...,...
361626,2023-08-08 18:29:00,10494,38,170
361627,2023-08-08 18:29:00,10427,16,170
361628,2023-08-08 18:29:00,10494,14,170
361629,2023-08-08 18:30:00,10442,49,170


Grouping the data frame by timestamps and line number because we need to process timestamps of different line numbers separately as the items can be manufactured from different line at the same time.

In [133]:
df_pro.rename(columns= {'timestamps utc': 'timestamps',
                        'id': 'itemid',
                        'duration in one minute(totalsecs)': 'totalsecs', 
                        'line number (160 or170)': 'slot'}, inplace=True)

Renaming columns into shorter, meaningful and usable names for preprocessing.

In [134]:
df_pro.head(3)

,timestamps,itemid,totalsecs,slot
0,2022-01-04 12:17:00,14,8,160
1,2022-01-04 12:46:00,103,7,160
2,2022-01-04 12:47:00,112,8,160


In [135]:
def merge_timestamps(df_p, threshold_seconds=180):
    df = df_p.copy()
    df['merged_itemid'] = df['itemid']
    df['merged_totalsecs'] = df['totalsecs']

    # Iterate through rows to merge subsequent observations
    for i in tqdm(range(1, len(df))):
        current_row = df.iloc[i]
        prev_row = df.iloc[i - 1]

        # Check if the slots are the same and the timestamp difference is below the threshold
        if (
            current_row['slot'] == prev_row['slot'] and
            (current_row['timestamps'] - prev_row['timestamps']).total_seconds() < threshold_seconds
        ):
            # Merge subsequent observations
            df.at[i, 'merged_itemid'] = prev_row['merged_itemid']
            df.at[i, 'merged_totalsecs'] = prev_row['merged_totalsecs'] + current_row['totalsecs']

    # Drop the unnecessary columns
    df.drop(columns=['itemid', 'totalsecs'], inplace=True)

    # Rename the merged columns
    df.rename(columns={'merged_itemid': 'itemid', 'merged_totalsecs': 'totalsecs'}, inplace=True)

    return df

In [136]:
df_processed = merge_timestamps(df_pro)

  0%|          | 0/361630 [00:00<?, ?it/s]

In [137]:
# df_processed.to_csv("clean_dataset.csv")
# df_processed = pd.read_csv('clean_dataset.csv').drop("Unnamed: 0", axis=1)

In [138]:
df_processed

,timestamps,slot,itemid,totalsecs
0,2022-01-04 12:17:00,160,14,8
1,2022-01-04 12:46:00,160,103,7
2,2022-01-04 12:47:00,160,103,15
3,2022-01-04 12:48:00,160,103,16
4,2022-01-04 12:49:00,160,103,24
...,...,...,...,...
361626,2023-08-08 18:29:00,170,6834,9967
361627,2023-08-08 18:29:00,170,6834,9983
361628,2023-08-08 18:29:00,170,6834,9997
361629,2023-08-08 18:30:00,170,6834,10046


In [139]:
df_processed.shape

(361631, 4)

Converting the datetime into suitable format to fed into model.
1. Converting to respective components.
2. Converting the whole datetime to seconds.

In [140]:
# df_processed['year'] = df_processed['timestamps'].dt.year
# df_processed['month'] = df_processed['timestamps'].dt.month
# df_processed['day'] = df_processed['timestamps'].dt.day
# df_processed['hour'] = df_processed['timestamps'].dt.hour
# df_processed['minute'] = df_processed['timestamps'].dt.minute
# df_processed['second'] = df_processed['timestamps'].dt.second

In [141]:
df_processed['timestamps_seconds'] = pd.to_datetime(df_processed['timestamps']).astype('int64') // 10**9
df_processed.head(3)

,timestamps,slot,itemid,totalsecs,timestamps_seconds
0,2022-01-04 12:17:00,160,14,8,1641298620
1,2022-01-04 12:46:00,160,103,7,1641300360
2,2022-01-04 12:47:00,160,103,15,1641300420


# Dummy Dataset Generation

Using Generative Adversial Network (GAN) model for dummy dataset generation.

In [142]:
class GANModel:
    def __init__(self, output_size,input_size=100, batch_size=64, epochs=1000, lr=0.002):
        # Hyperparameters
        self.input_size = input_size
        self.output_size = output_size
        self.batch_size = batch_size
        self.epochs = epochs
        self.lr = lr
        self.model = None
        self.gen = None
        self.dis = None

    # Generator definition
    def build_generator(self):
        model = Sequential()
        model.add(Dense(128, input_dim=self.input_size, activation='relu'))
        model.add(Dense(self.output_size, activation='linear'))
        return model

    # Discriminator definition
    def build_discriminator(self):
        model = Sequential()
        model.add(Dense(128, input_dim=self.output_size, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        return model

    def build_model(self):
        # Build and compile the discriminator
        discriminator = self.build_discriminator()
        discriminator.compile(optimizer=Adam(learning_rate=self.lr), loss='binary_crossentropy', metrics=['accuracy'])
        self.dis = discriminator

        # Build the generator
        generator = self.build_generator()
        self.gen = generator

        # Build the GAN model (stacking generator and discriminator)
        self.dis.trainable = False 
        gan_input = Input(shape=(self.input_size,))
        fake_data = self.gen(gan_input)
        gan_output = self.dis(fake_data)
        gan = Model(gan_input, gan_output)
        gan.compile(optimizer=Adam(learning_rate=self.lr), loss='binary_crossentropy', metrics=['accuracy'])
        self.model = gan
        
    def data_batch_generator(self, df, columns):
        
        num_batches = df.shape[0] // self.batch_size

        for epoch in range(num_batches):
            start_idx = epoch * self.batch_size
            end_idx = (epoch + 1) * self.batch_size

            batch_data = df.iloc[start_idx:end_idx][columns].values
            yield batch_data
    
    def train_model(self, cols, df):
        data_gen = self.data_batch_generator(df, cols)
        
        for epoch in range(self.epochs):
            
            noise = np.random.normal(0, 1, size=(self.batch_size, self.input_size))

            # Generate fake data using the generator
            generated_data = self.gen.predict(noise, verbose=0)

            real_data = next(data_gen)

            # Labels for real and fake data
            real_labels = np.ones((self.batch_size, 1))
            fake_labels = np.zeros((self.batch_size, 1))

            # Train discriminator on real and fake data
            d_loss_real = self.dis.train_on_batch(real_data, real_labels)
            d_loss_fake = self.dis.train_on_batch(generated_data, fake_labels)

            # Calculate total discriminator loss
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train generator
            noise = np.random.normal(0, 1, size=(self.batch_size, self.input_size))
            g_loss = self.model.train_on_batch(noise, real_labels)

            # Print progress
            if epoch % 100 == 0:
                print(f'Epoch {epoch}/{self.epochs}, D Loss: {d_loss[0]}, G Loss: {g_loss[0]}, G Accuracy: {g_loss[1]}')
                
    def generate_real_data(self, sample_size):
        noise = np.random.normal(0, 1, size=(sample_size, self.input_size))
        return self.gen.predict(noise)

In [143]:
gan = GANModel(output_size=len(train_cols))
gan.build_model()
gan.model.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 100)]             0         
                                                                 
 sequential_27 (Sequential)  (None, 4)                 13444     
                                                                 
 sequential_26 (Sequential)  (None, 1)                 769       
                                                                 
Total params: 14213 (55.52 KB)
Trainable params: 13444 (52.52 KB)
Non-trainable params: 769 (3.00 KB)
_________________________________________________________________


In [ ]:
gan.train_model(train_cols, df_processed)

Epoch 0/1000, D Loss: 0.32326841354370117, G Loss: 0.7710257768630981, G Accuracy: 0.21875
Epoch 100/1000, D Loss: 0.4198937714099884, G Loss: 0.6869375705718994, G Accuracy: 0.53125
Epoch 200/1000, D Loss: 5.20088005065918, G Loss: 6.375869270414114e-05, G Accuracy: 1.0
Epoch 300/1000, D Loss: 3.7786812782287598, G Loss: 0.006947527173906565, G Accuracy: 1.0
Epoch 400/1000, D Loss: 2.883057117462158, G Loss: 0.017524030059576035, G Accuracy: 1.0
Epoch 500/1000, D Loss: 2.705474853515625, G Loss: 0.022034533321857452, G Accuracy: 1.0


In [ ]:
def generate_real_data(sample_size):
    noise = np.random.normal(0, 1, size=(sample_size, 100))
    return gan.gen.predict(noise, verbose=1)

In [ ]:
df_gen = pd.DataFrame(generate_real_data(1000))

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(df_gen.iloc[:, -1], df_gen.iloc[:, -2], 'g--')
plt.show()

# Statistical Analysis

# Data Visualization and Analysis

In [ ]:
def plot_average_production_time(df):
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(df['timestamps'], df['totalsecs'], color="orange")
    plt.ylabel("Average time duration for an event")
    plt.xlabel("Date")
    plt.title("Date versus Average Time")
    plt.show()

In [ ]:
plot_average_production_time(df_processed)

The production time is much less in 2023 compared to 2022.